## 슈퍼마켓의 판매데이터를 전처리 

### 1.슈퍼마켓의 판매데이터를 전처리

In [1]:
import pandas as pd

In [2]:
data1 = pd.read_csv("슈퍼마켓/판매내역.csv", encoding="utf-8")
data2 = pd.read_csv("슈퍼마켓/공급자.csv", encoding="utf-8")
data3 = pd.read_csv("슈퍼마켓/상품정보.csv", encoding="cp949", sep="|")
data4 = pd.read_csv("슈퍼마켓/소분류.csv", encoding="cp949")
data5 = pd.read_csv("슈퍼마켓/중분류.csv", encoding="cp949")
data6 = pd.read_csv("슈퍼마켓/대분류.csv", encoding="cp949")

## 결측치나 데이터 타입을 변경하여 데이터 통합작업준비

### 2.작업전 데이터 정비(결측치 보정, 데이터 타입 변경)

In [3]:
# 필드 조정 null 제거 및 필드명 변경
data3.isnull().sum()

상품코드          0
상품명           4
수량        20968
분류1           4
분류2           4
등록일시,,        4
dtype: int64

In [4]:
data3 = data3.dropna(subset=["분류1", "분류2"])

In [5]:
data3.rename(columns={"분류1":"소분류코드"}, inplace=True)
data3.rename(columns={"분류2":'공급자코드'}, inplace=True)

In [6]:
data3

,상품코드,상품명,수량,소분류코드,공급자코드,"등록일시,,"
0,2100000626052,송염고은단치약130*3,NaN,20301.0,1042.0,"2021-01-26 18:01:54,,"
1,2100000626069,2080어린이딸기향80*2,3입,20101.0,6007.0,"2021-01-26 18:01:54,,"
2,2100000626076,LG토디앙황토특대/40p,40p,20402.0,1051.0,"2021-01-26 18:01:54,,"
3,2100000626083,참기름자반볶음번들,50g+50g,10301.0,9999.0,"2021-01-26 18:01:54,,"
4,2100000613007,할라페뇨슬라이스,540g,10903.0,9999.0,"2021-01-26 18:01:54,,"
...,...,...,...,...,...,...
34769,2100000111848,버/수퍼슬립레드보루,NaN,60102.0,7777.0,"2021-01-26 18:01:54,,"
34770,2100000111855,델)프/오렌지*2병,NaN,10201.0,5097.0,"2021-01-26 18:01:54,,"
34771,2100000106158,캡휴지걸이,NaN,20301.0,5014.0,"2021-01-26 18:01:54,,"
34772,2100000106165,커피향초중,NaN,20301.0,5014.0,"2021-01-26 18:01:54,,"


In [7]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34770 entries, 0 to 34773
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상품코드    34770 non-null  object 
 1   상품명     34770 non-null  object 
 2   수량      13806 non-null  object 
 3   소분류코드   34770 non-null  float64
 4   공급자코드   34770 non-null  float64
 5   등록일시,,  34770 non-null  object 
dtypes: float64(2), object(4)
memory usage: 1.9+ MB


In [8]:
#필드타입변경 float64 → int64
data3 = data3.astype({"소분류코드":"int64"})
data3 = data3.astype({"공급자코드":"int64"})

## 역정규화로 데이터를 하나로 합침

### 3.데이터 조인 및 정리

In [9]:
#데이터의 조인(inner join) 대분류 및 중분류
mdata = pd.merge(left = data5, right = data6, how="inner", on="대분류코드")

In [10]:
#데이터의 조인(inner join) (대분류+중분류)+소분류
classData = pd.merge(left=mdata, right=data4, how="inner", on="중분류코드")

In [11]:
#데이터의 조인(inner join) 분류코드 + 상품정보
product = pd.merge(left=classData, right=data3, how="inner", on="소분류코드")

In [12]:
#시각화에 사용되지 않는 필드 삭제
product = product.drop(["등록일시", "대분류코드", "중분류코드", "소분류코드", "등록일시,,", "수량"], axis="columns")

In [13]:
#데이터의 조인 (inner join) 상품정보 + 공급자
productDetail = pd.merge(left=product, right=data2, how="inner", on="공급자코드")

In [14]:
#시각화에 사용되지 않는 필드 삭제
productDetail = productDetail.drop(["공급자코드"], axis='columns')

In [15]:
productDetail

,중분류명,대분류명,소분류명,상품코드,상품명,공급자명
0,제과제빵,식품,"과자,껌,초코렛",2100000613182,꿀꽈배기90g*30입박스,농심라면
1,제과제빵,식품,"과자,껌,초코렛",2100000631957,농심쌀새우깡1상자,농심라면
2,제과제빵,식품,"과자,껌,초코렛",2100000631964,포스틱1상자,농심라면
3,제과제빵,식품,"과자,껌,초코렛",2100000613984,자갈치90g*20입박스,농심라면
4,제과제빵,식품,"과자,껌,초코렛",2100000594634,농심노래방새우깡박스,농심라면
...,...,...,...,...,...,...
5185,통조림(캔),식품,캔종류,2100000568062,오)후레쉬참치,오뚜기
5186,통조림(캔),식품,캔종류,2100000568079,오)오뚜기고등어,오뚜기
5187,기타식품,식품,기타식품류,2100000793396,오뚜기햄버그컵밥,오뚜기
5188,기타식품,식품,기타식품류,2100000821983,오뚜기참깨돈까스소스,오뚜기


In [16]:
productDetail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5190 entries, 0 to 5189
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   중분류명              5190 non-null   object
 1   대분류명              5190 non-null   object
 2   소분류명              5190 non-null   object
 3   상품코드              5190 non-null   object
 4   상품명               5190 non-null   object
 5   공급자명              5190 non-null   object
dtypes: object(6)
memory usage: 243.4+ KB


In [17]:
#데이터 타입 변경
data1 = data1.astype({"판매일시":"datetime64[ns]"})

In [18]:
#데이터 타입 변경
productDetail = productDetail.astype({"상품코드":"int64"})

In [19]:
productDetail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5190 entries, 0 to 5189
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   중분류명              5190 non-null   object
 1   대분류명              5190 non-null   object
 2   소분류명              5190 non-null   object
 3   상품코드              5190 non-null   int64 
 4   상품명               5190 non-null   object
 5   공급자명              5190 non-null   object
dtypes: int64(1), object(5)
memory usage: 243.4+ KB


In [20]:
#데이터의 조인(inner join) 상품데이터 + 판매데이터
salesData = pd.merge(left=productDetail, right=data1, how="inner", on="상품코드")

In [21]:
#시각화에 사용하지 않은 필드 삭제
salesData = salesData.drop(["상품코드"], axis="columns")

In [22]:
#새로운 데이터 판매년도를 생성함
salesData['판매년도'] = salesData["판매일시"].dt.year

In [23]:
salesData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227009 entries, 0 to 227008
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   중분류명              227009 non-null  object        
 1   대분류명              227009 non-null  object        
 2   소분류명              227009 non-null  object        
 3   상품명               227009 non-null  object        
 4   공급자명              227009 non-null  object        
 5   판매일시              227009 non-null  datetime64[ns]
 6   판매횟수              227009 non-null  int64         
 7   판매량               227009 non-null  int64         
 8   판매금액              227009 non-null  int64         
 9   판매년도              227009 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(3), object(5)
memory usage: 16.5+ MB


In [24]:
salesData.to_csv("판매정보.csv", encoding="utf-8-sig", index_label='순서')